In [1]:
# ===== Mount Google Drive =====
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# WITH CONTEXT

## Dataset Generation

In [2]:
# ================================
# Noisy Perfect V4: Dataset Builder
# ================================

# ===== Imports =====
import os, csv, random
import numpy as np
from tqdm import tqdm

# ===== Config =====
# I/O
LAYOUT_TXT  = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_layouts.txt"
OUT_DIR     = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_matrix_dataset_noisy_v4"
os.makedirs(OUT_DIR, exist_ok=True)

# Data size / split
NUM_TOTAL     = 10000       # number of total sequences
VAL_RATIO     = 0.10        # 10% validation
WINDOW_SIZE   = 4           # 4 trials -> 4*25 + '=>' + label
RNG_SEED      = 42          # reproducibility

# Noise / task
STIMULI_TOKENS  = ['U','D','L','R','A','B','C','E']
PLACEHOLDER     = '0'
CONGRUENCY_RATIO= 0.5         # probability of congruent trial
CORRUPTION_PROB = 0.10        # corruption prob. for U/D/L/R only

# Vocab
TOKENS = STIMULI_TOKENS + [PLACEHOLDER, '=>']
stoi = {tok:i for i,tok in enumerate(TOKENS)}
itos = {i:tok for tok,i in stoi.items()}

# ===== Seed =====
random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

# ===== Load symmetric layouts =====
SYMMETRIC_LAYOUTS = []
with open(LAYOUT_TXT, "r") as f:
    for line in f:
        s = line.strip()
        if not s:
            continue
        pos = tuple(map(int, s.split(',')))
        assert len(pos) == 5, f"Layout length != 5: {pos}"
        assert len(set(pos)) == 5, f"Duplicate positions: {pos}"
        assert all(0 <= p < 25 for p in pos), f"Invalid position: {pos}"
        SYMMETRIC_LAYOUTS.append(pos)

print(f"Loaded {len(SYMMETRIC_LAYOUTS)} layouts from file")

# ===== Helpers =====
def generate_centric_matrix_trial(layout, congruent=True):
    """
    Generate 5x5 (flattened to 25) matrix.
    - layout[2] is always the target
    - other positions are flankers
    - stimuli chosen from STIMULI_TOKENS
    """
    mat = [PLACEHOLDER] * 25
    target_token = random.choice(STIMULI_TOKENS)
    if congruent:
        flanker_token = target_token
    else:
        choices = [t for t in STIMULI_TOKENS if t != target_token]
        flanker_token = random.choice(choices)

    for i, p in enumerate(layout):
        mat[p] = target_token if i == 2 else flanker_token
    return mat, target_token, congruent

def corrupt_matrix_v4(matrix, prob=0.10):
    """
    V4 corruption: only applies to U/D/L/R tokens.
    Each is replaced with another direction (U/D/L/R) with given prob.
    """
    corrupted = matrix.copy()
    for i, tok in enumerate(corrupted):
        if tok in ['U','D','L','R'] and random.random() < prob:
            choices = [t for t in ['U','D','L','R'] if t != tok]
            corrupted[i] = random.choice(choices)
    return corrupted

def encode_sequence(trials, label_token):
    """
    Concatenate 4 trials (4*25 tokens) + '=>' + label,
    then convert to token IDs.
    """
    seq = []
    for t in trials:
        seq.extend(t)
    seq.append('=>')
    seq.append(label_token)
    return [stoi[t] for t in seq]

# ===== Generate sequences =====
def generate_sequence_matrix(window_size=4):
    trials, layouts, flags, labels = [], [], [], []
    for _ in range(window_size):
        layout = random.choice(SYMMETRIC_LAYOUTS)
        is_cong = (random.random() < CONGRUENCY_RATIO)
        mat, tgt, flag = generate_centric_matrix_trial(layout, congruent=is_cong)
        mat_cor = corrupt_matrix_v4(mat, prob=CORRUPTION_PROB)
        trials.append(mat_cor)
        layouts.append(layout)
        flags.append(flag)
        labels.append(tgt)
    return trials, labels[-1], layouts[-1], flags[-1]

def generate_matrix_dataset(num_sequences=10000, window_size=4):
    seqs, layout4, flags4 = [], [], []
    for _ in tqdm(range(num_sequences), desc="Generating V4 sequences"):
        trials, label_tok, layout_last, cong_last = generate_sequence_matrix(window_size)
        token_ids = encode_sequence(trials, label_tok)
        seqs.append(token_ids); layout4.append(layout_last); flags4.append(cong_last)
    return seqs, layout4, flags4

print("Generating dataset ...")
all_sequences, all_layouts, all_congruents = generate_matrix_dataset(
    num_sequences=NUM_TOTAL,
    window_size=WINDOW_SIZE
)

# ===== Shuffle & split (aligned) =====
N = len(all_sequences)
perm = np.random.RandomState(RNG_SEED).permutation(N)
all_sequences = [all_sequences[i] for i in perm]
all_layouts   = [all_layouts[i]   for i in perm]
all_congruents= [all_congruents[i]for i in perm]

split_idx = int(N * (1 - VAL_RATIO))
train_set = all_sequences[:split_idx]
val_set   = all_sequences[split_idx:]
train_layouts = all_layouts[:split_idx]
val_layouts   = all_layouts[split_idx:]
train_flags   = all_congruents[:split_idx]
val_flags     = all_congruents[split_idx:]

# ===== Save .npy =====
np.save(os.path.join(OUT_DIR, 'train.npy'), np.array(train_set, dtype=np.int32))
np.save(os.path.join(OUT_DIR, 'val.npy'),   np.array(val_set,   dtype=np.int32))
print(f"Saved .npy -> {OUT_DIR}")
print(f"train.npy: {len(train_set):,} | val.npy: {len(val_set):,}")

# ===== Save aligned meta (CSV) =====
import pandas as pd
def layout_to_str(tup5): return "-".join(map(str, tup5))

train_meta = pd.DataFrame({
    "trial4_layout":   [layout_to_str(t) for t in train_layouts],
    "is_congruent":    [int(f) for f in train_flags],
    "seq_len":         [len(s) for s in train_set],
})
val_meta = pd.DataFrame({
    "trial4_layout":   [layout_to_str(t) for t in val_layouts],
    "is_congruent":    [int(f) for f in val_flags],
    "seq_len":         [len(s) for s in val_set],
})

train_meta.to_csv(os.path.join(OUT_DIR, "train_meta.csv"), index=False)
val_meta.to_csv(os.path.join(OUT_DIR, "val_meta.csv"), index=False)
print("Saved aligned meta CSV (train_meta.csv, val_meta.csv)")

# ===== Save a combined CSV (optional, preview/debug) =====
combined_csv = os.path.join(OUT_DIR, "flanker_matrix_noisy_v4_preview.csv")
with open(combined_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["trial_1_4_text", "target_response_token_id", "trial4_layout", "is_congruent"])
    for token_ids, layout, flag in zip(all_sequences, all_layouts, all_congruents):
        text = " ".join([itos[i] for i in token_ids[:-2]])
        target_id = token_ids[-1]
        layout_str = layout_to_str(layout)
        writer.writerow([text, target_id, layout_str, int(flag)])
print(f"Preview CSV saved: {combined_csv}")

# ===== Save vocab =====
with open(os.path.join(OUT_DIR, "vocab.txt"), "w") as f:
    f.write(",".join(TOKENS))
print("Saved vocab.txt")

# ===== Quick sanity =====
print("\nSanity check:")
print(" - Example seq_len (should be 4*25+2 = 102):", val_meta["seq_len"].iloc[0] if len(val_meta) else "N/A")
print(" - Expected block_size for training: 102-1 = 101")
print("\n✅ DONE.")

Loaded 128 layouts from file
Generating dataset ...


Generating V4 sequences: 100%|██████████| 10000/10000 [00:00<00:00, 36873.75it/s]


Saved .npy -> /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_matrix_dataset_noisy_v4
train.npy: 9,000 | val.npy: 1,000
Saved aligned meta CSV (train_meta.csv, val_meta.csv)
Preview CSV saved: /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_matrix_dataset_noisy_v4/flanker_matrix_noisy_v4_preview.csv
Saved vocab.txt

Sanity check:
 - Example seq_len (should be 4*25+2 = 102): 102
 - Expected block_size for training: 102-1 = 101

✅ DONE.


## Training

In [3]:
# ===== Imports =====
import os
import sys
import torch
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm

# Add path so Colab can locate model.py (adjust as needed)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Flanker-GPT')
from model import GPT, GPTConfig

# ===== Random Seed (for reproducibility) =====
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# ===== Define Token Mapping (same as dataset script) =====
STIMULI_TOKENS = ['U', 'D', 'L', 'R', 'A', 'B', 'C', 'E']
PLACEHOLDER = '0'
SPECIAL_TOKENS = ['=>']
TOKENS = STIMULI_TOKENS + [PLACEHOLDER] + SPECIAL_TOKENS
stoi = {tok: i for i, tok in enumerate(TOKENS)}
itos = {i: tok for tok, i in stoi.items()}
vocab_size = len(stoi)

# ===== Paths =====
data_dir = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_matrix_dataset_noisy_v4"
out_dir = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/checkpoints"
os.makedirs(out_dir, exist_ok=True)

# ===== Training Settings =====
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 64
block_size = 101  # 25 tokens x 4 trials + 1 token (=>) + 1 target
max_iters = 2000
eval_interval = 200
learning_rate = 3e-4

# ===== Load Dataset =====
train_data = np.load(os.path.join(data_dir, 'train.npy'))
val_data = np.load(os.path.join(data_dir, 'val.npy'))

# ===== Get Batch =====
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = np.random.randint(len(data), size=batch_size)
    x = torch.tensor(np.stack([d[:-1] for d in data[ix]]), dtype=torch.long, device=device)
    y = torch.tensor([d[-1] for d in data[ix]], dtype=torch.long, device=device)
    return x, y

# ===== Create Model =====
config = GPTConfig(
    vocab_size=vocab_size,
    block_size=block_size,
    n_layer=4,
    n_head=4,
    n_embd=128
)
model = GPT(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# ===== Evaluate Loss (train & val) =====
@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = torch.zeros(10)
        for k in range(10):
            X, Y = get_batch(split)
            logits, _ = model(X)
            logits = logits[:, -1, :]  # Only the final token
            loss = F.cross_entropy(logits, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# ===== Training Loop =====
print("Training GPT-Flanker (Perfect Model with Symmetric Layouts)...")
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, _ = model(xb)
    logits = logits[:, -1, :]
    loss = F.cross_entropy(logits, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# ===== Save Model Checkpoint =====
ckpt_path = os.path.join(out_dir, "flanker_gpt_matrix_noisy_v4.pt")
torch.save(model.state_dict(), ckpt_path)
print(f"✅ Model saved to {ckpt_path}")

number of parameters: 0.79M
Training GPT-Flanker (Perfect Model with Symmetric Layouts)...
Step 0: train loss 2.3996, val loss 2.4089
Step 200: train loss 0.5263, val loss 0.5474
Step 400: train loss 0.2626, val loss 0.2540
Step 600: train loss 0.2826, val loss 0.2832
Step 800: train loss 0.2502, val loss 0.2469
Step 1000: train loss 0.2797, val loss 0.2306
Step 1200: train loss 0.2552, val loss 0.3127
Step 1400: train loss 0.2695, val loss 0.2122
Step 1600: train loss 0.2026, val loss 0.2481
Step 1800: train loss 0.1812, val loss 0.2592
Step 1999: train loss 0.2171, val loss 0.3679
✅ Model saved to /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/checkpoints/flanker_gpt_matrix_noisy_v4.pt


## Evaluation

In [4]:
# ===== Imports =====
import os, sys, csv, math
import torch
import numpy as np
import torch.nn.functional as F
import pandas as pd

# ===== Add model path =====
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Flanker-GPT')
from model import GPT, GPTConfig

# ===== Settings =====
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# V4 vocab (8 response tokens + 0 + =>)
vocab = ['U','D','L','R','A','B','C','E','0','=>']
itos = {i: ch for i, ch in enumerate(vocab)}
stoi = {ch: i for i, ch in enumerate(vocab)}
response_tokens = ['U','D','L','R','A','B','C','E']
response_token_ids = [stoi[t] for t in response_tokens]

# ===== Stopping-Rule Settings =====
delta = 3
max_samples = 100
restrict_to_response_tokens = True

# ===== Paths =====
base_dir      = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT"
data_dir      = f"{base_dir}/flanker_matrix_dataset_noisy_v4"
ckpt_path     = f"{base_dir}/checkpoints/flanker_gpt_matrix_noisy_v4.pt"
val_path      = os.path.join(data_dir, "val.npy")
val_meta_path = os.path.join(data_dir, "val_meta.csv")
output_csv    = f"{base_dir}/gpt_noisy_v4_val_predictions_stopping_delta{delta}_max{max_samples}.csv"

# ===== Load Validation Data (.npy) =====
val_data = np.load(val_path)
num_val = len(val_data)
block_size = val_data.shape[1] - 1  # robust
print(f"Loaded {num_val} validation samples | inferred block_size={block_size}")

# ===== Load aligned meta (val_meta.csv) =====
assert os.path.exists(val_meta_path), f"val_meta.csv not found: {val_meta_path}"
meta = pd.read_csv(val_meta_path)
assert len(meta) == num_val, f"val_meta rows ({len(meta)}) != VAL size ({num_val})"

# ===== Load Trained Model =====
config = GPTConfig(vocab_size=len(vocab), block_size=block_size, n_layer=4, n_head=4, n_embd=128)
model = GPT(config).to(device)
model.load_state_dict(torch.load(ckpt_path, map_location=device))
model.eval()
print("✅ Loaded Noisy GPT Model")

# ===== Utils =====
def calculate_entropy(prob_dist_np):
    return -float(np.sum(prob_dist_np * np.log(prob_dist_np + 1e-12)))

def sample_until_threshold(probs_full, delta=3, max_samples=100, restrict_ids=None):
    if restrict_ids is not None:
        sub = probs_full[restrict_ids]
        sub = sub / sub.sum()
        id_map = restrict_ids
        def draw():
            return id_map[torch.multinomial(sub, num_samples=1, replacement=True).item()]
    else:
        def draw():
            return torch.multinomial(probs_full, num_samples=1, replacement=True).item()

    counts = {}
    for s in range(1, max_samples + 1):
        tok = draw()
        counts[tok] = counts.get(tok, 0) + 1
        sorted_pairs = sorted(counts.items(), key=lambda kv: kv[1], reverse=True)
        top = sorted_pairs[0][1]
        second = sorted_pairs[1][1] if len(sorted_pairs) > 1 else 0
        gap = top - second
        if gap >= delta:
            return sorted_pairs[0][0], s, True, gap

    # fallback: argmax within restriction or full
    if restrict_ids is not None:
        winner_id = restrict_ids[torch.argmax(probs_full[restrict_ids]).item()]
    else:
        winner_id = torch.argmax(probs_full).item()

    if counts:
        sorted_pairs = sorted(counts.items(), key=lambda kv: kv[1], reverse=True)
        gap = sorted_pairs[0][1] - (sorted_pairs[1][1] if len(sorted_pairs) > 1 else 0)
    else:
        gap = 0
    return winner_id, max_samples, False, gap

# ===== Evaluate =====
results = []
correct = 0

for i in range(num_val):
    x = torch.tensor(val_data[i][:-1], dtype=torch.long, device=device).unsqueeze(0)
    true_id = int(val_data[i][-1])
    true_tok = itos[true_id]

    with torch.no_grad():
        logits, _ = model(x)
        probs = F.softmax(logits[:, -1, :], dim=-1).squeeze(0).cpu()

        winner_id, k_used, stopped, gap_at_stop = sample_until_threshold(
            probs_full=probs,
            delta=delta,
            max_samples=max_samples,
            restrict_ids=response_token_ids if restrict_to_response_tokens else None
        )

        pred_tok = itos[winner_id]
        is_correct = (winner_id == true_id)
        if is_correct:
            correct += 1

        confidence = float(probs[winner_id].item())
        prob_true  = float(probs[true_id].item())
        entropy    = calculate_entropy(probs.numpy())

    # meta (aligned)
    is_congruent  = int(meta.loc[i, "is_congruent"])
    trial4_layout = meta.loc[i, "trial4_layout"]  # "a-b-c-d-e"

    row = {
        "example": i + 1,
        "true_response": true_tok,
        "predicted_response": pred_tok,
        "is_correct": bool(is_correct),

        "confidence": round(confidence, 6),
        "prob_of_true_token": round(prob_true, 6),
        "entropy": round(entropy, 6),

        "k_samples": int(k_used),
        "stopped_by_delta": bool(stopped),
        "gap_at_stop": int(gap_at_stop),
        "delta": int(delta),
        "max_samples": int(max_samples),

        "is_congruent": is_congruent,
        "trial4_layout": trial4_layout,
    }

    for tok in response_tokens:
        row[f"{tok}_prob"] = round(float(probs[stoi[tok]].item()), 6)

    results.append(row)

# ===== Save CSV =====
with open(output_csv, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=list(results[0].keys()))
    writer.writeheader()
    writer.writerows(results)

acc = correct / num_val
print("\n✅ Stopping-rule evaluation complete.")
print(f"Accuracy = {acc * 100:.2f}% (Δ={delta}, max={max_samples})")
print(f"Saved to: {output_csv}")

Loaded 1000 validation samples | inferred block_size=101
number of parameters: 0.79M
✅ Loaded Noisy GPT Model

✅ Stopping-rule evaluation complete.
Accuracy = 90.10% (Δ=3, max=100)
Saved to: /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_val_predictions_stopping_delta3_max100.csv


### Evaluation Analysis

In [5]:
import pandas as pd
import numpy as np

# ==== Input CSV path ====
csv_path = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_val_predictions_stopping_delta3_max100.csv"

df = pd.read_csv(csv_path)
print("Rows:", len(df))
print("Columns:", list(df.columns))

# Required columns for analysis
required = ["is_congruent","k_samples","stopped_by_delta","gap_at_stop","is_correct","entropy","confidence","delta","max_samples"]
missing = [c for c in required if c not in df.columns]
print("Missing required columns:", missing)

# ==== Basic accuracy ====
acc_overall = df["is_correct"].mean()
print(f"\nAccuracy overall: {acc_overall*100:.2f}%")

# ==== k_samples summary ====
def summarize_k(x):
    q = x.quantile([0, .25, .5, .75, .95, .99]).to_dict()
    return {
        "count": int(x.count()),
        "min": float(x.min()),
        "mean": float(x.mean()),
        "std": float(x.std(ddof=1)),
        "p25": float(q[0.25]),
        "p50": float(q[0.5]),
        "p75": float(q[0.75]),
        "p95": float(q[0.95]),
        "p99": float(q[0.99]),
        "max": float(x.max()),
    }

print("\n== k_samples summary (overall) ==")
print(summarize_k(df["k_samples"]))

print("\n== k_samples summary by congruency ==")
for flag, g in df.groupby("is_congruent"):
    lab = "congruent" if flag==1 else "incongruent"
    print(f"\n[{lab}] n={len(g)}")
    print("acc:", round(g["is_correct"].mean()*100,2), "%")
    print("entropy mean:", round(g["entropy"].mean(),4))
    print(summarize_k(g["k_samples"]))

# ==== stopping statistics ====
stop_rate = df["stopped_by_delta"].mean()
hit_max = (df["k_samples"] >= df["max_samples"]).mean()
print(f"\nStopped by threshold (rate): {stop_rate*100:.2f}%")
print(f"Hit max_samples (rate): {hit_max*100:.2f}%")

# ==== correlation checks ====
def safe_corr(a, b):
    if a.std(ddof=1)==0 or b.std(ddof=1)==0:
        return np.nan
    return float(np.corrcoef(a, b)[0,1])

corr_k_entropy = safe_corr(df["k_samples"], df["entropy"])
corr_k_one_minus_conf = safe_corr(df["k_samples"], 1 - df["confidence"])
print(f"\nCorr(k, entropy): {corr_k_entropy:.3f}")
print(f"Corr(k, 1-confidence): {corr_k_one_minus_conf:.3f}")

# ==== predicted response distribution (quick check) ====
if "predicted_response" in df.columns:
    print("\nPredicted response counts (top 10):")
    print(df["predicted_response"].value_counts().head(10))

# ==== save first 100 rows (for easy sharing/debug) ====
sample_path = csv_path.replace(".csv", "_SAMPLE100.csv")
df.head(100).to_csv(sample_path, index=False)
print("\nSaved sample to:", sample_path)

Rows: 1000
Columns: ['example', 'true_response', 'predicted_response', 'is_correct', 'confidence', 'prob_of_true_token', 'entropy', 'k_samples', 'stopped_by_delta', 'gap_at_stop', 'delta', 'max_samples', 'is_congruent', 'trial4_layout', 'U_prob', 'D_prob', 'L_prob', 'R_prob', 'A_prob', 'B_prob', 'C_prob', 'E_prob']
Missing required columns: []

Accuracy overall: 90.10%

== k_samples summary (overall) ==
{'count': 1000, 'min': 3.0, 'mean': 3.878, 'std': 3.215813654031796, 'p25': 3.0, 'p50': 3.0, 'p75': 3.0, 'p95': 9.0, 'p99': 19.0, 'max': 48.0}

== k_samples summary by congruency ==

[incongruent] n=483
acc: 85.92 %
entropy mean: 0.3217
{'count': 483, 'min': 3.0, 'mean': 4.15527950310559, 'std': 3.6271664115260465, 'p25': 3.0, 'p50': 3.0, 'p75': 3.0, 'p95': 11.0, 'p99': 16.62000000000006, 'max': 48.0}

[congruent] n=517
acc: 94.0 %
entropy mean: 0.1774
{'count': 517, 'min': 3.0, 'mean': 3.618955512572534, 'std': 2.7555209681939647, 'p25': 3.0, 'p50': 3.0, 'p75': 3.0, 'p95': 6.0, 'p99': 

### Shared RT Parameters

In [6]:
### **You should run the evaluation to export files for all models first before running this cell**

# ===========================================
# Shared RT Parameters (Pooled)
# - Computes theta and dispersion from human RT and model k-samples
# - In-memory only (no saving to disk)
# ===========================================
import os
import numpy as np
import pandas as pd

# ---------- CONFIG ----------
HUMAN_META_TRAIN = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/human_dataset/train_meta.csv"
HUMAN_META_VAL   = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/human_dataset/val_meta.csv"

MODEL_PRED_FILES = [
    # context versions
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_perfect_pf_val_predictions_8resp_delta3_max100.csv",
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v1_val_predictions_stopping_delta3_max100.csv",
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v2_val_predictions_stopping_delta3_max100.csv",
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v3_val_predictions_stopping_delta3_max100.csv",
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_val_predictions_stopping_delta3_max100.csv",

    # no-context versions
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_perfect_pf_nocontext_val_predictions_8resp_delta3_max100.csv",
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v1_nocontext_val_predictions_stopping_delta3_max100.csv",
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v2_nocontext_val_predictions_stopping_delta3_max100.csv",
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v3_nocontext_val_predictions_stopping_delta3_max100.csv",
    "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_nocontext_val_predictions_stopping_delta3_max100.csv",
]

# constants
t0 = 300.0                # ms (non-decision time)
TRIM_LO, TRIM_HI = 150, 3000   # human RT trimming range
DISPERSION_FALLBACK = 2.25     # fallback if estimation fails
DISPERSION_FILE_MASK = None    # optionally restrict dispersion calibration to subset of models


# ---------- Helpers ----------
def _maybe_trim(s, lo, hi):
    if lo is not None: s = s[s >= lo]
    if hi is not None: s = s[s <= hi]
    return s

def load_human_stats(train_meta_path, val_meta_path, t0, trim_lo=None, trim_hi=None):
    """Load and summarize human RT data."""
    meta = pd.concat([pd.read_csv(train_meta_path), pd.read_csv(val_meta_path)], ignore_index=True)
    assert {"trial4_response_time", "trial4_is_congruent"}.issubset(meta.columns), \
        "human meta must contain trial4_response_time and trial4_is_congruent"
    rt = meta["trial4_response_time"].dropna().astype(float)
    cong = meta.loc[rt.index, "trial4_is_congruent"].astype(int)
    rt = _maybe_trim(rt, trim_lo, trim_hi); cong = cong.loc[rt.index]
    p_con = float((cong == 1).mean()); p_inc = 1.0 - p_con
    mu_con = float(rt[cong == 1].mean()); mu_inc = float(rt[cong == 0].mean())
    mu_pool = p_con * mu_con + p_inc * mu_inc
    var_rt_minus_t0 = float((rt - t0).var(ddof=0))
    return dict(p_con=p_con, p_inc=p_inc, mu_con=mu_con, mu_inc=mu_inc,
                mu_pool=mu_pool, var_rt_minus_t0=var_rt_minus_t0,
                overall_mean=float(rt.mean()), overall_std=float(rt.std(ddof=0)), n_used=int(len(rt)))

def load_k_stats(pred_files, mask=None):
    """Load k_samples from prediction CSV files."""
    ks = []
    for fp in pred_files:
        bn = os.path.basename(fp)
        assert "SAMPLE" not in bn and "_RT_mapped_" not in bn, f"Do not use sample or RT-mapped files: {bn}"
        if mask is not None and not any(m.lower() in bn.lower() for m in mask):
            continue
        df = pd.read_csv(fp)
        assert "k_samples" in df.columns, f"{bn} does not contain column k_samples"
        ks.append(df["k_samples"].astype(float).values)
    if not ks:
        raise ValueError("No matching prediction files found")
    k_all = np.concatenate(ks, axis=0)
    return dict(k_mean=float(k_all.mean()), k_var=float(k_all.var(ddof=0)), n=int(k_all.size))

def theta_from(mu_pool_human, t0, k_mean_pool):
    """Compute shared theta from human mean RT and model mean k."""
    if k_mean_pool <= 0:
        raise ValueError("k_mean_pool must be > 0")
    return float((mu_pool_human - t0) / k_mean_pool)

def dispersion_from(var_rt_minus_t0, theta_shared, k_mean_pool, k_var_pool):
    """Compute dispersion parameter (>=1)."""
    # Var(RT - t0) = theta^2 * ( E[k]/disp + Var(k) )
    denom = (var_rt_minus_t0 / (theta_shared ** 2)) - k_var_pool
    if denom <= 0: return None
    return float(k_mean_pool / denom)


# ---------- Compute ----------
human = load_human_stats(HUMAN_META_TRAIN, HUMAN_META_VAL, t0, TRIM_LO, TRIM_HI)
k_all = load_k_stats(MODEL_PRED_FILES, mask=None)

theta_shared = theta_from(human["mu_pool"], t0, k_all["k_mean"])

if DISPERSION_FILE_MASK:
    k_for_disp = load_k_stats(MODEL_PRED_FILES, mask=DISPERSION_FILE_MASK)
else:
    k_for_disp = k_all

dispersion_shared = dispersion_from(human["var_rt_minus_t0"], theta_shared, k_for_disp["k_mean"], k_for_disp["k_var"])
dispersion_note = "pooled_estimated"
if dispersion_shared is None:
    dispersion_shared = float(DISPERSION_FALLBACK)
    dispersion_note = "used_fallback"

# ---------- Summary ----------
print("=== Shared RT Parameters (Plan B / pooled) ===")
print(f"t0 = {t0:.2f} ms")
print(f"theta_shared = {theta_shared:.4f} ms/sample")
print(f"dispersion_shared = {dispersion_shared:.4f}  ({dispersion_note})")
print("\n--- Human (trimmed) ---")
print(f"n = {human['n_used']:,}")
print(f"p_con = {human['p_con']:.4f} | p_inc = {human['p_inc']:.4f}")
print(f"mu_con = {human['mu_con']:.2f} ms | mu_inc = {human['mu_inc']:.2f} ms | mu_pool = {human['mu_pool']:.2f} ms")
print(f"overall_mean = {human['overall_mean']:.2f} | overall_std = {human['overall_std']:.2f}")
print(f"Var(RT - t0) = {human['var_rt_minus_t0']:.2f}")
print("\n--- k pooled (all models) ---")
print(f"k_mean = {k_all['k_mean']:.6f} | k_var = {k_all['k_var']:.6f} | n = {k_all['n']:,}")

# Final usable variables:
#   t0, theta_shared, dispersion_shared
#   human (dict), k_all (dict), k_for_disp (dict)

=== Shared RT Parameters (Plan B / pooled) ===
t0 = 300.00 ms
theta_shared = 150.7292 ms/sample
dispersion_shared = 2.2500  (used_fallback)

--- Human (trimmed) ---
n = 1,070,284
p_con = 0.5014 | p_inc = 0.4986
mu_con = 750.51 ms | mu_inc = 826.34 ms | mu_pool = 788.32 ms
overall_mean = 788.32 | overall_std = 204.96
Var(RT - t0) = 42008.21

--- k pooled (all models) ---
k_mean = 3.239700 | k_var = 2.463644 | n = 10,000


### RT Mapping (Shared Parameters)

In [7]:
# ===== Map RT with SHARED parameters (no per-condition fitting) =====
import pandas as pd
import numpy as np
import os

# ---- INPUT ----
in_csv = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_val_predictions_stopping_delta3_max100.csv"

# ---- SHARED PARAMS (fixed across models & conditions) ----
try:
    t0          # already defined upstream?
    theta_shared
    dispersion_shared
    dispersion = float(dispersion_shared)
except NameError:
    # Fixed parameters (Plan B values)
    t0 = 300.0             # non-decision time (ms)
    theta_shared = 151.0837  # ms per sample
    dispersion = 2.25        # controls variance spread

t0_jitter_sd = 0.0           # set >0 to add Gaussian noise to t0
use_erlang = True            # True=Gamma/Erlang, False=deterministic linear

# ---- LOAD ----
df = pd.read_csv(in_csv)
assert "k_samples" in df.columns, "CSV must contain column 'k_samples'"

# ---- MAP RT (using shared theta) ----
rng = np.random.default_rng(20250809)

def draw_rt_row(row):
    k = int(max(1, row["k_samples"]))
    t0_eff = t0 + (rng.normal(0, t0_jitter_sd) if t0_jitter_sd > 0 else 0.0)
    if not use_erlang:
        return float(t0_eff + k * theta_shared)
    # Erlang/Gamma with dispersion control:
    # mean = k * theta_shared, var = theta_shared^2 * (k / dispersion)
    shape = k * dispersion
    scale = theta_shared / dispersion
    return float(t0_eff + rng.gamma(shape=shape, scale=scale))

df["rt_ms"] = df.apply(draw_rt_row, axis=1)

# (Optional) clip to human-reported RT range
# df["rt_ms"] = df["rt_ms"].clip(150, 3000)

# ---- SAVE ----
suffix = "_RT_mapped_SHARED"
out_csv = os.path.splitext(in_csv)[0] + f"{suffix}.csv"
df.to_csv(out_csv, index=False)

# ---- QUICK SUMMARY ----
def summarize(s):
    d = s.describe(percentiles=[.25, .5, .75, .9, .95, .99])
    return {k: float(d[k]) for k in d.index}

overall_pred = summarize(df["rt_ms"])

means_by_c = {}
gap_pred = float("nan")
if "is_congruent" in df.columns:
    means_by_c = df.groupby("is_congruent")["rt_ms"].mean().to_dict()
    gap_pred = means_by_c.get(0, np.nan) - means_by_c.get(1, np.nan)

print("\n=== Predicted RT (SHARED params) ===")
print(f"t0={t0:.1f} ms | theta_shared={theta_shared:.4f} | dispersion={dispersion:.4f}")
print("Overall:", overall_pred)
if means_by_c:
    print(f"Means by congruency: inc={means_by_c.get(0, np.nan):.2f} | con={means_by_c.get(1, np.nan):.2f} | gap={gap_pred:.2f} ms")
print("Saved:", out_csv)


=== Predicted RT (SHARED params) ===
t0=300.0 ms | theta_shared=150.7292 | dispersion=2.2500
Overall: {'count': 1000.0, 'mean': 885.9668700160096, 'std': 496.29159248284, 'min': 387.26590013520786, '25%': 650.1012751684259, '50%': 771.6546055856979, '75%': 953.4420766604627, '90%': 1226.0397353390374, '95%': 1521.3667981556578, '99%': 3236.031760654691, 'max': 6659.602116799598}
Means by congruency: inc=928.59 | con=846.15 | gap=82.45 ms
Saved: /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_val_predictions_stopping_delta3_max100_RT_mapped_SHARED.csv


### Congruency Effect Analysis

In [8]:
# ===== Analyze Congruency Effect for NOISY V4 =====
# Run after evaluation (stopping rule) + RT mapping
# Requires column "rt_ms" in the input CSV

import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt

# ==== INPUT ====
base_dir = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT"
pred_csv = f"{base_dir}/gpt_noisy_v4_val_predictions_stopping_delta3_max100_RT_mapped_PERCOND.csv"
out_dir  = os.path.join(base_dir, "congruency_effect_noisy_v4")
os.makedirs(out_dir, exist_ok=True)

# (Optional) Human benchmark RTs for comparison
mu_con_human = 751.31
mu_inc_human = 825.40

# ==== LOAD ====
df = pd.read_csv(pred_csv)
req = {"is_congruent","rt_ms","is_correct","entropy","confidence","k_samples"}
missing = list(req - set(df.columns))
assert not missing, f"Missing columns: {missing}"

# ==== BASIC SUMMARIES ====
def summ(x: pd.Series):
    d = x.describe(percentiles=[.25,.5,.75,.9,.95,.99])
    return {k: float(d[k]) for k in d.index}

overall_rt = summ(df["rt_ms"])
by_c = df.groupby("is_congruent")["rt_ms"].mean().to_dict()
gap_ms = by_c.get(0, np.nan) - by_c.get(1, np.nan)

acc_overall = float(df["is_correct"].mean()*100)
acc_by_c = (df.groupby("is_congruent")["is_correct"]
              .mean()
              .rename({0:"incongruent",1:"congruent"})*100)

ent_by_c = df.groupby("is_congruent")["entropy"].mean().rename({0:"incongruent",1:"congruent"})
k_by_c   = df.groupby("is_congruent")["k_samples"].mean().rename({0:"incongruent",1:"congruent"})

def cohens_d(a, b):
    a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
    if len(a)<2 or len(b)<2: return np.nan
    m1, m2 = a.mean(), b.mean()
    s1, s2 = a.std(ddof=1), b.std(ddof=1)
    sp = np.sqrt(((len(a)-1)*s1*s1 + (len(b)-1)*s2*s2)/(len(a)+len(b)-2))
    return (m1-m2)/sp if sp>0 else np.nan

rt_inc = df.loc[df["is_congruent"]==0, "rt_ms"].values
rt_con = df.loc[df["is_congruent"]==1, "rt_ms"].values
d_eff  = cohens_d(rt_inc, rt_con)

print("=== Congruency Effect (NOISY V4) ===")
print("RT overall:", overall_rt)
print(f"RT means by congruency (ms): inc={by_c.get(0, np.nan):.2f} | con={by_c.get(1, np.nan):.2f} | gap={gap_ms:.2f}")
print(f"Cohen's d (inc - con): {d_eff:.3f}")
print(f"Accuracy overall: {acc_overall:.2f}% | by congruency (%):\n{acc_by_c.to_string()}")
print(f"Entropy mean by congruency:\n{ent_by_c.to_string()}")
print(f"k_samples mean by congruency:\n{k_by_c.to_string()}")

# --- Human comparison (optional) ---
print("\n--- vs Human targets (optional) ---")
human_gap = mu_inc_human - mu_con_human
print(f"Human means: con={mu_con_human:.2f} | inc={mu_inc_human:.2f} | gap={human_gap:.2f} ms")
print(f"Model deltas: Δcon={by_c.get(1, np.nan)-mu_con_human:.2f} | Δinc={by_c.get(0, np.nan)-mu_inc_human:.2f} | Δgap={gap_ms - human_gap:.2f} ms")

# ==== SAVE TABLE ====
rows = [
    {"metric":"RT_overall", **overall_rt},
    {"metric":"RT_mean_congruent", "mean": by_c.get(1, np.nan)},
    {"metric":"RT_mean_incongruent", "mean": by_c.get(0, np.nan)},
    {"metric":"RT_gap_inc_minus_con", "gap_ms": gap_ms},
    {"metric":"Cohens_d_inc_minus_con", "d": d_eff},
    {"metric":"Accuracy_overall_%", "value": acc_overall},
    {"metric":"Accuracy_congruent_%", "value": acc_by_c.get("congruent", np.nan)},
    {"metric":"Accuracy_incongruent_%", "value": acc_by_c.get("incongruent", np.nan)},
    {"metric":"Entropy_mean_congruent", "value": ent_by_c.get("congruent", np.nan)},
    {"metric":"Entropy_mean_incongruent", "value": ent_by_c.get("incongruent", np.nan)},
    {"metric":"k_mean_congruent", "value": k_by_c.get("congruent", np.nan)},
    {"metric":"k_mean_incongruent", "value": k_by_c.get("incongruent", np.nan)},
]
out_csv = os.path.join(out_dir, os.path.basename(pred_csv).replace(".csv","_congruency_stats.csv"))
pd.DataFrame(rows).to_csv(out_csv, index=False)

# ==== PLOTS ====
plt.figure(figsize=(6,4))
plt.hist(df.loc[df["is_congruent"]==1, "rt_ms"], bins=60, alpha=0.6, label="congruent")
plt.hist(df.loc[df["is_congruent"]==0, "rt_ms"], bins=60, alpha=0.6, label="incongruent")
plt.xlabel("RT (ms)"); plt.ylabel("Count"); plt.title("RT by Congruency (Histogram)")
plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(out_dir,"rt_hist_by_congruency.png"), dpi=160); plt.close()

def ecdf(x):
    xs = np.sort(x); ys = np.arange(1, len(xs)+1)/len(xs); return xs, ys
x1,y1 = ecdf(rt_con); x2,y2 = ecdf(rt_inc)
plt.figure(figsize=(6,4))
plt.plot(x1,y1,label="congruent"); plt.plot(x2,y2,label="incongruent")
plt.xlabel("RT (ms)"); plt.ylabel("CDF"); plt.title("RT CDF by Congruency")
plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(out_dir,"rt_cdf_by_congruency.png"), dpi=160); plt.close()

plt.figure(figsize=(5,4))
plt.boxplot([rt_con, rt_inc], labels=["congruent","incongruent"])
plt.ylabel("RT (ms)"); plt.title("RT Boxplot by Congruency")
plt.tight_layout()
plt.savefig(os.path.join(out_dir,"rt_box_by_congruency.png"), dpi=160); plt.close()

print("\nSaved:", out_csv)
print("Plots saved to:", out_dir)

=== Congruency Effect (NOISY V4) ===
RT overall: {'count': 1000.0, 'mean': 790.6254443200005, 'std': 357.0511834055209, 'min': 375.257204525453, '25%': 599.0292543124782, '50%': 704.6843215877027, '75%': 862.1016777780608, '90%': 1081.1792063202465, '95%': 1348.3390652867597, '99%': 2503.77781521526, 'max': 3812.335922740273}
RT means by congruency (ms): inc=828.39 | con=755.34 | gap=73.05
Cohen's d (inc - con): 0.206
Accuracy overall: 90.30% | by congruency (%):
is_congruent
incongruent    86.128364
congruent      94.197292
Entropy mean by congruency:
is_congruent
incongruent    0.321699
congruent      0.177376
k_samples mean by congruency:
is_congruent
incongruent    4.155280
congruent      3.471954

--- vs Human targets (optional) ---
Human means: con=751.31 | inc=825.40 | gap=74.09 ms
Model deltas: Δcon=4.03 | Δinc=2.99 | Δgap=-1.04 ms

Saved: /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/congruency_effect_noisy_v4/gpt_noisy_v4_val_predictions_stopping_delta3_max100_RT_mapped_

/tmp/ipython-input-3620120575.py:103: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot([rt_con, rt_inc], labels=["congruent","incongruent"])


# WITHOUT CONTEXT

## Dataset Generation

In [9]:
# ================================
# Noisy Perfect V4: Dataset Builder
# ================================

# ===== Imports =====
import os, csv, random
import numpy as np
from tqdm import tqdm

# ===== Config =====
# I/O
LAYOUT_TXT  = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_layouts.txt"
OUT_DIR = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_matrix_dataset_noisy_v4_nocontext" #*
os.makedirs(OUT_DIR, exist_ok=True)

# Data size / split
NUM_TOTAL     = 10000       # number of total sequences
VAL_RATIO     = 0.10        # 10% validation
WINDOW_SIZE   = 1           # 1 trial -> 4*25 + '=>' + label #*
RNG_SEED      = 42          # reproducibility

# Noise / task
STIMULI_TOKENS  = ['U','D','L','R','A','B','C','E']
PLACEHOLDER     = '0'
CONGRUENCY_RATIO= 0.5         # probability of congruent trial
CORRUPTION_PROB = 0.10        # corruption prob. for U/D/L/R only

# Vocab
TOKENS = STIMULI_TOKENS + [PLACEHOLDER, '=>']
stoi = {tok:i for i,tok in enumerate(TOKENS)}
itos = {i:tok for tok,i in stoi.items()}

# ===== Seed =====
random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

# ===== Load symmetric layouts =====
SYMMETRIC_LAYOUTS = []
with open(LAYOUT_TXT, "r") as f:
    for line in f:
        s = line.strip()
        if not s:
            continue
        pos = tuple(map(int, s.split(',')))
        assert len(pos) == 5, f"Layout length != 5: {pos}"
        assert len(set(pos)) == 5, f"Duplicate positions: {pos}"
        assert all(0 <= p < 25 for p in pos), f"Invalid position: {pos}"
        SYMMETRIC_LAYOUTS.append(pos)

print(f"Loaded {len(SYMMETRIC_LAYOUTS)} layouts from file")

# ===== Helpers =====
def generate_centric_matrix_trial(layout, congruent=True):
    """
    Generate 5x5 (flattened to 25) matrix.
    - layout[2] is always the target
    - other positions are flankers
    - stimuli chosen from STIMULI_TOKENS
    """
    mat = [PLACEHOLDER] * 25
    target_token = random.choice(STIMULI_TOKENS)
    if congruent:
        flanker_token = target_token
    else:
        choices = [t for t in STIMULI_TOKENS if t != target_token]
        flanker_token = random.choice(choices)

    for i, p in enumerate(layout):
        mat[p] = target_token if i == 2 else flanker_token
    return mat, target_token, congruent

def corrupt_matrix_v4(matrix, prob=0.10):
    """
    V4 corruption: only applies to U/D/L/R tokens.
    Each is replaced with another direction (U/D/L/R) with given prob.
    """
    corrupted = matrix.copy()
    for i, tok in enumerate(corrupted):
        if tok in ['U','D','L','R'] and random.random() < prob:
            choices = [t for t in ['U','D','L','R'] if t != tok]
            corrupted[i] = random.choice(choices)
    return corrupted

def encode_sequence(trials, label_token):
    """
    Concatenate 4 trials (4*25 tokens) + '=>' + label,
    then convert to token IDs.
    """
    seq = []
    for t in trials:
        seq.extend(t)
    seq.append('=>')
    seq.append(label_token)
    return [stoi[t] for t in seq]

# ===== Generate sequences =====
def generate_sequence_matrix(window_size=4):
    trials, layouts, flags, labels = [], [], [], []
    for _ in range(window_size):
        layout = random.choice(SYMMETRIC_LAYOUTS)
        is_cong = (random.random() < CONGRUENCY_RATIO)
        mat, tgt, flag = generate_centric_matrix_trial(layout, congruent=is_cong)
        mat_cor = corrupt_matrix_v4(mat, prob=CORRUPTION_PROB)
        trials.append(mat_cor)
        layouts.append(layout)
        flags.append(flag)
        labels.append(tgt)
    return trials, labels[-1], layouts[-1], flags[-1]

def generate_matrix_dataset(num_sequences=10000, window_size=4):
    seqs, layout4, flags4 = [], [], []
    for _ in tqdm(range(num_sequences), desc="Generating V4 sequences"):
        trials, label_tok, layout_last, cong_last = generate_sequence_matrix(window_size)
        token_ids = encode_sequence(trials, label_tok)
        seqs.append(token_ids); layout4.append(layout_last); flags4.append(cong_last)
    return seqs, layout4, flags4

print("Generating dataset ...")
all_sequences, all_layouts, all_congruents = generate_matrix_dataset(
    num_sequences=NUM_TOTAL,
    window_size=WINDOW_SIZE
)

# ===== Shuffle & split (aligned) =====
N = len(all_sequences)
perm = np.random.RandomState(RNG_SEED).permutation(N)
all_sequences = [all_sequences[i] for i in perm]
all_layouts   = [all_layouts[i]   for i in perm]
all_congruents= [all_congruents[i]for i in perm]

split_idx = int(N * (1 - VAL_RATIO))
train_set = all_sequences[:split_idx]
val_set   = all_sequences[split_idx:]
train_layouts = all_layouts[:split_idx]
val_layouts   = all_layouts[split_idx:]
train_flags   = all_congruents[:split_idx]
val_flags     = all_congruents[split_idx:]

# ===== Save .npy =====
np.save(os.path.join(OUT_DIR, 'train.npy'), np.array(train_set, dtype=np.int32))
np.save(os.path.join(OUT_DIR, 'val.npy'),   np.array(val_set,   dtype=np.int32))
print(f"Saved .npy -> {OUT_DIR}")
print(f"train.npy: {len(train_set):,} | val.npy: {len(val_set):,}")

# ===== Save aligned meta (CSV) =====
import pandas as pd
def layout_to_str(tup5): return "-".join(map(str, tup5))

train_meta = pd.DataFrame({
    "trial4_layout":   [layout_to_str(t) for t in train_layouts],
    "is_congruent":    [int(f) for f in train_flags],
    "seq_len":         [len(s) for s in train_set],
})
val_meta = pd.DataFrame({
    "trial4_layout":   [layout_to_str(t) for t in val_layouts],
    "is_congruent":    [int(f) for f in val_flags],
    "seq_len":         [len(s) for s in val_set],
})

train_meta.to_csv(os.path.join(OUT_DIR, "train_meta.csv"), index=False)
val_meta.to_csv(os.path.join(OUT_DIR, "val_meta.csv"), index=False)
print("Saved aligned meta CSV (train_meta.csv, val_meta.csv)")

# ===== Save a combined CSV (optional, preview/debug) =====
combined_csv = os.path.join(OUT_DIR, "flanker_matrix_noisy_v4_nocontext_preview.csv") #*
with open(combined_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["trial_1_4_text", "target_response_token_id", "trial4_layout", "is_congruent"])
    for token_ids, layout, flag in zip(all_sequences, all_layouts, all_congruents):
        text = " ".join([itos[i] for i in token_ids[:-2]])
        target_id = token_ids[-1]
        layout_str = layout_to_str(layout)
        writer.writerow([text, target_id, layout_str, int(flag)])
print(f"Preview CSV saved: {combined_csv}")

# ===== Save vocab =====
with open(os.path.join(OUT_DIR, "vocab.txt"), "w") as f:
    f.write(",".join(TOKENS))
print("Saved vocab.txt")

# ===== Quick sanity =====
print("\nSanity check:")
print(" - Example seq_len (should be 1*25+2 = 27):", val_meta["seq_len"].iloc[0] if len(val_meta) else "N/A") #*
print(" - Expected block_size for training:", "27-1 = 26") #*
print("\n✅ DONE.")

Loaded 128 layouts from file
Generating dataset ...


Generating V4 sequences: 100%|██████████| 10000/10000 [00:00<00:00, 82213.37it/s]

Saved .npy -> /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_matrix_dataset_noisy_v4_nocontext
train.npy: 9,000 | val.npy: 1,000


Saved aligned meta CSV (train_meta.csv, val_meta.csv)
Preview CSV saved: /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_matrix_dataset_noisy_v4_nocontext/flanker_matrix_noisy_v4_nocontext_preview.csv
Saved vocab.txt

Sanity check:
 - Example seq_len (should be 1*25+2 = 27): 27
 - Expected block_size for training: 27-1 = 26

✅ DONE.


## Training

In [10]:
# ===== Imports =====
import os
import sys
import torch
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm

# Add path so Colab can locate model.py (adjust as needed)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Flanker-GPT')
from model import GPT, GPTConfig

# ===== Random Seed (for reproducibility) =====
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# ===== Define Token Mapping (same as dataset script) =====
STIMULI_TOKENS = ['U', 'D', 'L', 'R', 'A', 'B', 'C', 'E']
PLACEHOLDER = '0'
SPECIAL_TOKENS = ['=>']
TOKENS = STIMULI_TOKENS + [PLACEHOLDER] + SPECIAL_TOKENS
stoi = {tok: i for i, tok in enumerate(TOKENS)}
itos = {i: tok for tok, i in stoi.items()}
vocab_size = len(stoi)

# ===== Paths =====
data_dir = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/flanker_matrix_dataset_noisy_v4_nocontext" #*
out_dir = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/checkpoints"
os.makedirs(out_dir, exist_ok=True)

# ===== Training Settings =====
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 64
block_size = 26  # 25 tokens x 1 trial + 1 token (=>) + 1 target #*
max_iters = 2000
eval_interval = 200
learning_rate = 3e-4

# ===== Load Dataset =====
train_data = np.load(os.path.join(data_dir, 'train.npy'))
val_data = np.load(os.path.join(data_dir, 'val.npy'))

# ===== Get Batch =====
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = np.random.randint(len(data), size=batch_size)
    x = torch.tensor(np.stack([d[:-1] for d in data[ix]]), dtype=torch.long, device=device)
    y = torch.tensor([d[-1] for d in data[ix]], dtype=torch.long, device=device)
    return x, y

# ===== Create Model =====
config = GPTConfig(
    vocab_size=vocab_size,
    block_size=block_size,
    n_layer=4,
    n_head=4,
    n_embd=128
)
model = GPT(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# ===== Evaluate Loss (train & val) =====
@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = torch.zeros(10)
        for k in range(10):
            X, Y = get_batch(split)
            logits, _ = model(X)
            logits = logits[:, -1, :]  # Only the final token
            loss = F.cross_entropy(logits, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# ===== Training Loop =====
print("🚀 Training GPT-Flanker (Perfect Model with Symmetric Layouts)...")
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, _ = model(xb)
    logits = logits[:, -1, :]
    loss = F.cross_entropy(logits, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# ===== Save Model Checkpoint =====
ckpt_path = os.path.join(out_dir, "flanker_gpt_matrix_noisy_v4_nocontext.pt") #*
torch.save(model.state_dict(), ckpt_path)
print(f"✅ Model saved to {ckpt_path}")

number of parameters: 0.79M
🚀 Training GPT-Flanker (Perfect Model with Symmetric Layouts)...
Step 0: train loss 2.5113, val loss 2.5116
Step 200: train loss 0.3174, val loss 0.3598
Step 400: train loss 0.1896, val loss 0.2496
Step 600: train loss 0.2821, val loss 0.3031
Step 800: train loss 0.2116, val loss 0.2476
Step 1000: train loss 0.2232, val loss 0.2337
Step 1200: train loss 0.2028, val loss 0.2057
Step 1400: train loss 0.2033, val loss 0.3218
Step 1600: train loss 0.2191, val loss 0.2077
Step 1800: train loss 0.2273, val loss 0.2013
Step 1999: train loss 0.1855, val loss 0.2101
✅ Model saved to /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/checkpoints/flanker_gpt_matrix_noisy_v4_nocontext.pt


## Evaluation

In [ ]:
# ===== Imports =====
import os, sys, csv, math
import torch
import numpy as np
import torch.nn.functional as F
import pandas as pd

# ===== Add model path =====
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Flanker-GPT')
from model import GPT, GPTConfig

# ===== Settings =====
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# V4 vocab (8 response tokens + 0 + =>)
vocab = ['U','D','L','R','A','B','C','E','0','=>']
itos = {i: ch for i, ch in enumerate(vocab)}
stoi = {ch: i for i, ch in enumerate(vocab)}
response_tokens = ['U','D','L','R','A','B','C','E']
response_token_ids = [stoi[t] for t in response_tokens]

# ===== Stopping-Rule Settings =====
delta = 3
max_samples = 100
restrict_to_response_tokens = True

# ===== Paths =====
base_dir      = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT"
data_dir      = f"{base_dir}/flanker_matrix_dataset_noisy_v4_nocontext" #*
ckpt_path     = f"{base_dir}/checkpoints/flanker_gpt_matrix_noisy_v4_nocontext.pt" #*
val_path      = os.path.join(data_dir, "val.npy")
val_meta_path = os.path.join(data_dir, "val_meta.csv")
output_csv    = f"{base_dir}/gpt_noisy_v4_nocontext_val_predictions_stopping_delta{delta}_max{max_samples}.csv" #*

# ===== Load Validation Data (.npy) =====
val_data = np.load(val_path)
num_val = len(val_data)
block_size = val_data.shape[1] - 1  # robust
print(f"Loaded {num_val} validation samples | inferred block_size={block_size}")

# ===== Load aligned meta (val_meta.csv) =====
assert os.path.exists(val_meta_path), f"val_meta.csv not found: {val_meta_path}"
meta = pd.read_csv(val_meta_path)
assert len(meta) == num_val, f"val_meta rows ({len(meta)}) != VAL size ({num_val})"

# ===== Load Trained Model =====
config = GPTConfig(vocab_size=len(vocab), block_size=block_size, n_layer=4, n_head=4, n_embd=128)
model = GPT(config).to(device)
model.load_state_dict(torch.load(ckpt_path, map_location=device))
model.eval()
print("✅ Loaded Noisy GPT Model")

# ===== Utils =====
def calculate_entropy(prob_dist_np):
    return -float(np.sum(prob_dist_np * np.log(prob_dist_np + 1e-12)))

def sample_until_threshold(probs_full, delta=3, max_samples=100, restrict_ids=None):
    if restrict_ids is not None:
        sub = probs_full[restrict_ids]
        sub = sub / sub.sum()
        id_map = restrict_ids
        def draw():
            return id_map[torch.multinomial(sub, num_samples=1, replacement=True).item()]
    else:
        def draw():
            return torch.multinomial(probs_full, num_samples=1, replacement=True).item()

    counts = {}
    for s in range(1, max_samples + 1):
        tok = draw()
        counts[tok] = counts.get(tok, 0) + 1
        sorted_pairs = sorted(counts.items(), key=lambda kv: kv[1], reverse=True)
        top = sorted_pairs[0][1]
        second = sorted_pairs[1][1] if len(sorted_pairs) > 1 else 0
        gap = top - second
        if gap >= delta:
            return sorted_pairs[0][0], s, True, gap

    # fallback: argmax within restriction or full
    if restrict_ids is not None:
        winner_id = restrict_ids[torch.argmax(probs_full[restrict_ids]).item()]
    else:
        winner_id = torch.argmax(probs_full).item()

    if counts:
        sorted_pairs = sorted(counts.items(), key=lambda kv: kv[1], reverse=True)
        gap = sorted_pairs[0][1] - (sorted_pairs[1][1] if len(sorted_pairs) > 1 else 0)
    else:
        gap = 0
    return winner_id, max_samples, False, gap

# ===== Evaluate =====
results = []
correct = 0

for i in range(num_val):
    x = torch.tensor(val_data[i][:-1], dtype=torch.long, device=device).unsqueeze(0)
    true_id = int(val_data[i][-1])
    true_tok = itos[true_id]

    with torch.no_grad():
        logits, _ = model(x)
        probs = F.softmax(logits[:, -1, :], dim=-1).squeeze(0).cpu()

        winner_id, k_used, stopped, gap_at_stop = sample_until_threshold(
            probs_full=probs,
            delta=delta,
            max_samples=max_samples,
            restrict_ids=response_token_ids if restrict_to_response_tokens else None
        )

        pred_tok = itos[winner_id]
        is_correct = (winner_id == true_id)
        if is_correct:
            correct += 1

        confidence = float(probs[winner_id].item())
        prob_true  = float(probs[true_id].item())
        entropy    = calculate_entropy(probs.numpy())

    # meta (aligned)
    is_congruent  = int(meta.loc[i, "is_congruent"])
    trial4_layout = meta.loc[i, "trial4_layout"]  # "a-b-c-d-e"

    row = {
        "example": i + 1,
        "true_response": true_tok,
        "predicted_response": pred_tok,
        "is_correct": bool(is_correct),

        "confidence": round(confidence, 6),
        "prob_of_true_token": round(prob_true, 6),
        "entropy": round(entropy, 6),

        "k_samples": int(k_used),
        "stopped_by_delta": bool(stopped),
        "gap_at_stop": int(gap_at_stop),
        "delta": int(delta),
        "max_samples": int(max_samples),

        "is_congruent": is_congruent,
        "trial4_layout": trial4_layout,
    }

    for tok in response_tokens:
        row[f"{tok}_prob"] = round(float(probs[stoi[tok]].item()), 6)

    results.append(row)

# ===== Save CSV =====
with open(output_csv, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=list(results[0].keys()))
    writer.writeheader()
    writer.writerows(results)

acc = correct / num_val
print("\n✅ Stopping-rule evaluation complete.")
print(f"Accuracy = {acc * 100:.2f}% (Δ={delta}, max={max_samples})")
print(f"Saved to: {output_csv}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📊 Loaded 1000 validation samples | inferred block_size=26
number of parameters: 0.79M
✅ Loaded Noisy GPT Model

✅ Stopping-rule evaluation complete.
🎯 Accuracy = 90.90% (Δ=3, max=100)
📄 Saved to: /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_nocontext_val_predictions_stopping_delta3_max100.csv


### Evaluation Analysis

In [11]:
import pandas as pd
import numpy as np

# ==== Input CSV path ====
csv_path = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_nocontext_val_predictions_stopping_delta3_max100.csv" #*

df = pd.read_csv(csv_path)
print("Rows:", len(df))
print("Columns:", list(df.columns))

# Required columns for analysis
required = ["is_congruent","k_samples","stopped_by_delta","gap_at_stop","is_correct","entropy","confidence","delta","max_samples"]
missing = [c for c in required if c not in df.columns]
print("Missing required columns:", missing)

# ==== Basic accuracy ====
acc_overall = df["is_correct"].mean()
print(f"\nAccuracy overall: {acc_overall*100:.2f}%")

# ==== k_samples summary ====
def summarize_k(x):
    q = x.quantile([0, .25, .5, .75, .95, .99]).to_dict()
    return {
        "count": int(x.count()),
        "min": float(x.min()),
        "mean": float(x.mean()),
        "std": float(x.std(ddof=1)),
        "p25": float(q[0.25]),
        "p50": float(q[0.5]),
        "p75": float(q[0.75]),
        "p95": float(q[0.95]),
        "p99": float(q[0.99]),
        "max": float(x.max()),
    }

print("\n== k_samples summary (overall) ==")
print(summarize_k(df["k_samples"]))

print("\n== k_samples summary by congruency ==")
for flag, g in df.groupby("is_congruent"):
    lab = "congruent" if flag==1 else "incongruent"
    print(f"\n[{lab}] n={len(g)}")
    print("acc:", round(g["is_correct"].mean()*100,2), "%")
    print("entropy mean:", round(g["entropy"].mean(),4))
    print(summarize_k(g["k_samples"]))

# ==== stopping statistics ====
stop_rate = df["stopped_by_delta"].mean()
hit_max = (df["k_samples"] >= df["max_samples"]).mean()
print(f"\nStopped by threshold (rate): {stop_rate*100:.2f}%")
print(f"Hit max_samples (rate): {hit_max*100:.2f}%")

# ==== correlation checks ====
def safe_corr(a, b):
    if a.std(ddof=1)==0 or b.std(ddof=1)==0:
        return np.nan
    return float(np.corrcoef(a, b)[0,1])

corr_k_entropy = safe_corr(df["k_samples"], df["entropy"])
corr_k_one_minus_conf = safe_corr(df["k_samples"], 1 - df["confidence"])
print(f"\nCorr(k, entropy): {corr_k_entropy:.3f}")
print(f"Corr(k, 1-confidence): {corr_k_one_minus_conf:.3f}")

# ==== predicted response distribution (quick check) ====
if "predicted_response" in df.columns:
    print("\nPredicted response counts (top 10):")
    print(df["predicted_response"].value_counts().head(10))

# ==== save first 100 rows (for easy sharing/debug) ====
sample_path = csv_path.replace(".csv", "_SAMPLE100.csv")
df.head(100).to_csv(sample_path, index=False)
print("\nSaved sample to:", sample_path)

Rows: 1000
Columns: ['example', 'true_response', 'predicted_response', 'is_correct', 'confidence', 'prob_of_true_token', 'entropy', 'k_samples', 'stopped_by_delta', 'gap_at_stop', 'delta', 'max_samples', 'is_congruent', 'trial4_layout', 'U_prob', 'D_prob', 'L_prob', 'R_prob', 'A_prob', 'B_prob', 'C_prob', 'E_prob']
Missing required columns: []

Accuracy overall: 90.90%

== k_samples summary (overall) ==
{'count': 1000, 'min': 3.0, 'mean': 3.648, 'std': 2.5104578065361953, 'p25': 3.0, 'p50': 3.0, 'p75': 3.0, 'p95': 7.0, 'p99': 14.009999999999991, 'max': 34.0}

== k_samples summary by congruency ==

[incongruent] n=490
acc: 88.98 %
entropy mean: 0.2343
{'count': 490, 'min': 3.0, 'mean': 3.816326530612245, 'std': 2.8805352645892346, 'p25': 3.0, 'p50': 3.0, 'p75': 3.0, 'p95': 8.0, 'p99': 15.550000000000068, 'max': 34.0}

[congruent] n=510
acc: 92.75 %
entropy mean: 0.1379
{'count': 510, 'min': 3.0, 'mean': 3.4862745098039216, 'std': 2.0843715969005694, 'p25': 3.0, 'p50': 3.0, 'p75': 3.0, '

### RT Mapping (Shared Parameters)

In [12]:
# ===== Map RT with SHARED parameters (no per-condition fitting) =====
import pandas as pd
import numpy as np
import os

# ---- INPUT ----
in_csv = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_nocontext_val_predictions_stopping_delta3_max100.csv" #*

# ---- SHARED PARAMS (fixed across models & conditions) ----
try:
    t0          # already defined upstream?
    theta_shared
    dispersion_shared
    dispersion = float(dispersion_shared)
except NameError:
    # Fixed parameters (Plan B values)
    t0 = 300.0             # non-decision time (ms)
    theta_shared = 151.0837  # ms per sample
    dispersion = 2.25        # controls variance spread

t0_jitter_sd = 0.0           # set >0 to add Gaussian noise to t0
use_erlang = True            # True=Gamma/Erlang, False=deterministic linear

# ---- LOAD ----
df = pd.read_csv(in_csv)
assert "k_samples" in df.columns, "CSV must contain column 'k_samples'"

# ---- MAP RT (using shared theta) ----
rng = np.random.default_rng(20250809)

def draw_rt_row(row):
    k = int(max(1, row["k_samples"]))
    t0_eff = t0 + (rng.normal(0, t0_jitter_sd) if t0_jitter_sd > 0 else 0.0)
    if not use_erlang:
        return float(t0_eff + k * theta_shared)
    # Erlang/Gamma with dispersion control:
    # mean = k * theta_shared, var = theta_shared^2 * (k / dispersion)
    shape = k * dispersion
    scale = theta_shared / dispersion
    return float(t0_eff + rng.gamma(shape=shape, scale=scale))

df["rt_ms"] = df.apply(draw_rt_row, axis=1)

# (Optional) clip to human-reported RT range
# df["rt_ms"] = df["rt_ms"].clip(150, 3000)

# ---- SAVE ----
suffix = "_RT_mapped_SHARED"
out_csv = os.path.splitext(in_csv)[0] + f"{suffix}.csv"
df.to_csv(out_csv, index=False)

# ---- QUICK SUMMARY ----
def summarize(s):
    d = s.describe(percentiles=[.25, .5, .75, .9, .95, .99])
    return {k: float(d[k]) for k in d.index}

overall_pred = summarize(df["rt_ms"])

means_by_c = {}
gap_pred = float("nan")
if "is_congruent" in df.columns:
    means_by_c = df.groupby("is_congruent")["rt_ms"].mean().to_dict()
    gap_pred = means_by_c.get(0, np.nan) - means_by_c.get(1, np.nan)

print("\n=== Predicted RT (SHARED params) ===")
print(f"t0={t0:.1f} ms | theta_shared={theta_shared:.4f} | dispersion={dispersion:.4f}")
print("Overall:", overall_pred)
if means_by_c:
    print(f"Means by congruency: inc={means_by_c.get(0, np.nan):.2f} | con={means_by_c.get(1, np.nan):.2f} | gap={gap_pred:.2f} ms")
print("Saved:", out_csv)


=== Predicted RT (SHARED params) ===
t0=300.0 ms | theta_shared=150.7292 | dispersion=2.2500
Overall: {'count': 1000.0, 'mean': 855.9313718202543, 'std': 434.1600379619752, 'min': 387.26590013520786, '25%': 644.3007290533053, '50%': 756.2154460754302, '75%': 913.9250377274489, '90%': 1144.9118576157098, '95%': 1441.7245467095404, '99%': 2845.193252957569, 'max': 5010.362053458899}
Means by congruency: inc=889.15 | con=824.02 | gap=65.13 ms
Saved: /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_nocontext_val_predictions_stopping_delta3_max100_RT_mapped_SHARED.csv


### Congruency Effect Analysis

In [13]:
# ===== Analyze Congruency Effect for NOISY V4 =====
# Run after evaluation (stopping rule) + RT mapping
# Requires column "rt_ms" in the input CSV

import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt

# ==== INPUT ====
base_dir = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT"
pred_csv = f"{base_dir}/gpt_noisy_v4_nocontext_val_predictions_stopping_delta3_max100_RT_mapped_PERCOND_nocontext.csv" #*
out_dir  = os.path.join(base_dir, "congruency_effect_noisy_v4_nocontext") #*
os.makedirs(out_dir, exist_ok=True)

# (Optional) Human benchmark RTs for comparison
mu_con_human = 751.31
mu_inc_human = 825.40

# ==== LOAD ====
df = pd.read_csv(pred_csv)
req = {"is_congruent","rt_ms","is_correct","entropy","confidence","k_samples"}
missing = list(req - set(df.columns))
assert not missing, f"Missing columns: {missing}"

# ==== BASIC SUMMARIES ====
def summ(x: pd.Series):
    d = x.describe(percentiles=[.25,.5,.75,.9,.95,.99])
    return {k: float(d[k]) for k in d.index}

overall_rt = summ(df["rt_ms"])
by_c = df.groupby("is_congruent")["rt_ms"].mean().to_dict()
gap_ms = by_c.get(0, np.nan) - by_c.get(1, np.nan)

acc_overall = float(df["is_correct"].mean()*100)
acc_by_c = (df.groupby("is_congruent")["is_correct"]
              .mean()
              .rename({0:"incongruent",1:"congruent"})*100)

ent_by_c = df.groupby("is_congruent")["entropy"].mean().rename({0:"incongruent",1:"congruent"})
k_by_c   = df.groupby("is_congruent")["k_samples"].mean().rename({0:"incongruent",1:"congruent"})

def cohens_d(a, b):
    a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
    if len(a)<2 or len(b)<2: return np.nan
    m1, m2 = a.mean(), b.mean()
    s1, s2 = a.std(ddof=1), b.std(ddof=1)
    sp = np.sqrt(((len(a)-1)*s1*s1 + (len(b)-1)*s2*s2)/(len(a)+len(b)-2))
    return (m1-m2)/sp if sp>0 else np.nan

rt_inc = df.loc[df["is_congruent"]==0, "rt_ms"].values
rt_con = df.loc[df["is_congruent"]==1, "rt_ms"].values
d_eff  = cohens_d(rt_inc, rt_con)

print("=== Congruency Effect (NOISY V4) ===")
print("RT overall:", overall_rt)
print(f"RT means by congruency (ms): inc={by_c.get(0, np.nan):.2f} | con={by_c.get(1, np.nan):.2f} | gap={gap_ms:.2f}")
print(f"Cohen's d (inc - con): {d_eff:.3f}")
print(f"Accuracy overall: {acc_overall:.2f}% | by congruency (%):\n{acc_by_c.to_string()}")
print(f"Entropy mean by congruency:\n{ent_by_c.to_string()}")
print(f"k_samples mean by congruency:\n{k_by_c.to_string()}")

# --- Human comparison (optional) ---
print("\n--- vs Human targets (optional) ---")
human_gap = mu_inc_human - mu_con_human
print(f"Human means: con={mu_con_human:.2f} | inc={mu_inc_human:.2f} | gap={human_gap:.2f} ms")
print(f"Model deltas: Δcon={by_c.get(1, np.nan)-mu_con_human:.2f} | Δinc={by_c.get(0, np.nan)-mu_inc_human:.2f} | Δgap={gap_ms - human_gap:.2f} ms")

# ==== SAVE TABLE ====
rows = [
    {"metric":"RT_overall", **overall_rt},
    {"metric":"RT_mean_congruent", "mean": by_c.get(1, np.nan)},
    {"metric":"RT_mean_incongruent", "mean": by_c.get(0, np.nan)},
    {"metric":"RT_gap_inc_minus_con", "gap_ms": gap_ms},
    {"metric":"Cohens_d_inc_minus_con", "d": d_eff},
    {"metric":"Accuracy_overall_%", "value": acc_overall},
    {"metric":"Accuracy_congruent_%", "value": acc_by_c.get("congruent", np.nan)},
    {"metric":"Accuracy_incongruent_%", "value": acc_by_c.get("incongruent", np.nan)},
    {"metric":"Entropy_mean_congruent", "value": ent_by_c.get("congruent", np.nan)},
    {"metric":"Entropy_mean_incongruent", "value": ent_by_c.get("incongruent", np.nan)},
    {"metric":"k_mean_congruent", "value": k_by_c.get("congruent", np.nan)},
    {"metric":"k_mean_incongruent", "value": k_by_c.get("incongruent", np.nan)},
]
out_csv = os.path.join(out_dir, os.path.basename(pred_csv).replace(".csv","_congruency_stats.csv"))
pd.DataFrame(rows).to_csv(out_csv, index=False)

# ==== PLOTS ====
plt.figure(figsize=(6,4))
plt.hist(df.loc[df["is_congruent"]==1, "rt_ms"], bins=60, alpha=0.6, label="congruent")
plt.hist(df.loc[df["is_congruent"]==0, "rt_ms"], bins=60, alpha=0.6, label="incongruent")
plt.xlabel("RT (ms)"); plt.ylabel("Count"); plt.title("RT by Congruency (Histogram)")
plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(out_dir,"rt_hist_by_congruency.png"), dpi=160); plt.close()

def ecdf(x):
    xs = np.sort(x); ys = np.arange(1, len(xs)+1)/len(xs); return xs, ys
x1,y1 = ecdf(rt_con); x2,y2 = ecdf(rt_inc)
plt.figure(figsize=(6,4))
plt.plot(x1,y1,label="congruent"); plt.plot(x2,y2,label="incongruent")
plt.xlabel("RT (ms)"); plt.ylabel("CDF"); plt.title("RT CDF by Congruency")
plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(out_dir,"rt_cdf_by_congruency.png"), dpi=160); plt.close()

plt.figure(figsize=(5,4))
plt.boxplot([rt_con, rt_inc], labels=["congruent","incongruent"])
plt.ylabel("RT (ms)"); plt.title("RT Boxplot by Congruency")
plt.tight_layout()
plt.savefig(os.path.join(out_dir,"rt_box_by_congruency.png"), dpi=160); plt.close()

print("\nSaved:", out_csv)
print("Plots saved to:", out_dir)

=== Congruency Effect (NOISY V4) ===
RT overall: {'count': 1000.0, 'mean': 793.2000160809132, 'std': 387.9329449126726, 'min': 376.88145603307305, '25%': 606.0311696653972, '50%': 703.1217579246248, '75%': 849.8043183245418, '90%': 1067.1250649588906, '95%': 1340.7458880160875, '99%': 2494.3997565612817, 'max': 4602.305980798192}
RT means by congruency (ms): inc=838.11 | con=750.05 | gap=88.06
Cohen's d (inc - con): 0.228
Accuracy overall: 90.90% | by congruency (%):
is_congruent
incongruent    88.979592
congruent      92.745098
Entropy mean by congruency:
is_congruent
incongruent    0.234321
congruent      0.137877
k_samples mean by congruency:
is_congruent
incongruent    3.816327
congruent      3.486275

--- vs Human targets (optional) ---
Human means: con=751.31 | inc=825.40 | gap=74.09 ms
Model deltas: Δcon=-1.26 | Δinc=12.71 | Δgap=13.97 ms

Saved: /content/drive/MyDrive/Colab Notebooks/Flanker-GPT/congruency_effect_noisy_v4_nocontext/gpt_noisy_v4_nocontext_val_predictions_stoppin

/tmp/ipython-input-1445173287.py:103: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot([rt_con, rt_inc], labels=["congruent","incongruent"])


# Erlang Distribution

Quantiles (clipped):
Human Logs      : {'p10': 592.0, 'p50': 746.0, 'p90': 1027.0}
Probabilistic Corruption (PC) Model (ctx) : {'p10': 562.07472528881, 'p50': 768.6821572350588, 'p90': 1196.7341401922308}
Probabilistic Corruption (PC) Model (no-ctx): {'p10': 555.3099154449997, 'p50': 753.6962335359217, 'p90': 1129.0880102582455}


In [14]:
# === RT distributions (clipped) — Human vs 1 Model (context & no-context) ===
# Provide paths for 3 inputs: human train/val meta, and 2 model CSVs (ctx & no-ctx)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ---------- PATHS (fill in accordingly) ----------
HUMAN_META_TRAIN = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/human_dataset/train_meta.csv"   # << path to human train meta
HUMAN_META_VAL   = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/human_dataset/val_meta.csv"     # << path to human val meta

MODEL_CTX_CSV    = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_val_predictions_stopping_delta3_max100_RT_mapped_SHARED.csv"     # << path to model (context)
MODEL_NOCTX_CSV  = "/content/drive/MyDrive/Colab Notebooks/Flanker-GPT/gpt_noisy_v4_nocontext_val_predictions_stopping_delta3_max100_RT_mapped_SHARED.csv"  # << path to model (no-context)

CLIP_LO, CLIP_HI = 150, 3000  # clipping range (same as human baseline)

# ---------- helpers ----------
def clip_series(s, lo, hi):
    return s[(s >= lo) & (s <= hi)]

def load_human_rt(train_meta_path, val_meta_path, lo, hi):
    meta = pd.concat([pd.read_csv(train_meta_path), pd.read_csv(val_meta_path)], ignore_index=True)
    assert "trial4_response_time" in meta.columns, "Human meta must contain column: trial4_response_time"
    rt = meta["trial4_response_time"].dropna().astype(float)
    rt = clip_series(rt, lo, hi)
    return rt.values

def load_model_rt(csv_path, lo, hi):
    df = pd.read_csv(csv_path)
    assert "rt_ms" in df.columns, f"{csv_path} must contain column: rt_ms"
    rt = clip_series(df["rt_ms"].astype(float), lo, hi)
    return rt.values

# ---------- load ----------
human_rt  = load_human_rt(HUMAN_META_TRAIN, HUMAN_META_VAL, CLIP_LO, CLIP_HI)
model_ctx = load_model_rt(MODEL_CTX_CSV,   CLIP_LO, CLIP_HI)
model_nc  = load_model_rt(MODEL_NOCTX_CSV, CLIP_LO, CLIP_HI)

# ---------- plot (PDF via histogram density) ----------
plt.figure(figsize=(8,5))
plt.hist(human_rt,  bins=60, range=(CLIP_LO,CLIP_HI), density=True, histtype="step", linewidth=1.5, label="Human Logs (Baseline)")
plt.hist(model_ctx, bins=60, range=(CLIP_LO,CLIP_HI), density=True, histtype="step", linewidth=1.2, label="Probabilistic Corruption (PC) Model (ctx)")
plt.hist(model_nc,  bins=60, range=(CLIP_LO,CLIP_HI), density=True, histtype="step", linewidth=1.2, label="Probabilistic Corruption (PC) Model (no-ctx)")
plt.xlabel(f"RT (ms) — clipped to [{CLIP_LO}, {CLIP_HI}]")
plt.ylabel("Density")
plt.title("RT distributions (Erlang-mapped) vs Human (clipped)")
plt.legend()
plt.tight_layout()
plt.show()

# ---------- quick summary (quantiles) ----------
def qtxt(a):
    return dict(p10=float(np.percentile(a,10)), p50=float(np.percentile(a,50)), p90=float(np.percentile(a,90)))

print("Quantiles (clipped):")
print("Human Logs      :", qtxt(human_rt))
print("Probabilistic Corruption (PC) Model (ctx) :", qtxt(model_ctx))
print("Probabilistic Corruption (PC) Model (no-ctx):", qtxt(model_nc))